Adding everything from Step-1.ipynb

In [39]:
import numpy
import pandas
import torch
import seaborn as sns
import matplotlib.pyplot as plt


import numpy
import pandas
import torch
import seaborn as sns
import matplotlib.pyplot as plt

In [40]:
import pandas as pd
import numpy as np

# Read in the data

## Read in joke id and joke text
jester_joke_data = pd.read_csv('datasets/jester_items.csv')

## Read in joke ratings
jester_joke_ratings = pd.read_csv('datasets/jester_ratings.csv')

## Read reddit jokes and unify the format for the joke data
reddit_jokes = pd.read_json('datasets/reddit_jokes.json')
reddit_jokes['jokeId'] = reddit_jokes.id

## TODO: Add this dataframe to the jester_joke_data dataframe and update the index
reddit_jokes['jokeText'] = reddit_jokes['title'] + '\n' + reddit_jokes['body']
reddit_joke_data = reddit_jokes[['jokeId', 'jokeText']]

## Read stupidstuff.json and unify the format for the joke data
stupidstuff_jokes = pd.read_json('datasets/stupidstuff_shuffled.json')

In [41]:
user_ids = [0, 1, 2]

user_ratings_files = [ 'datasets/ratings_USF100' + str(user_id) + '.csv' for user_id in user_ids ]

user_ratings = pd.concat([pd.read_csv(user_ratings_file) for user_ratings_file in user_ratings_files])

In [42]:
# We create new columns for all the jokes accoring to the user ratings

unique_user_ids = user_ratings['user_id'].unique()

for index, row in user_ratings.iterrows():

    user_num = row['user_id']
    joke_id = row['joke_id']

    joke_ratings = user_ratings[(user_ratings['joke_id'] == joke_id)]

    for user_id in unique_user_ids:
        ratings = joke_ratings[joke_ratings['user_id'] == user_id]['rating'].values
        if len(ratings) > 0:
            rating = int(ratings[0])
        else:
            rating = np.nan
        user_ratings.loc[index, (user_id + "_rating")] = rating

In [43]:
user_ratings.drop_duplicates(subset='joke_id', inplace=True)

user_ratings.drop(['user_id', 'rating'], axis=1, inplace=True)

In [44]:
# print(user_ratings)

# user_ratings.joke_id.unique()

def add_source_column(row):
    joke_id = row['joke_id']

    if joke_id.startswith('RE'):
        temp_joke_id = joke_id[2:]
        row['source'] = "reddit"
        row['datasource_joke_id'] = temp_joke_id
        # user_ratings.loc[index, 'source'] = "reddit"
        # user_ratings.loc[index, 'datasource_joke_id'] = temp_joke_id

    elif joke_id.startswith('JE'):
        temp_joke_id = joke_id[2:]
        row['source'] = "jester"
        row['datasource_joke_id'] = temp_joke_id

    elif joke_id.startswith('SS'):
        temp_joke_id = joke_id[2:]
        row['source'] = "stupidstuff"
        row['datasource_joke_id'] = temp_joke_id

    return row


user_ratings = user_ratings.apply(add_source_column, axis=1)

print(user_ratings.isna().sum())

na_user_ratings = user_ratings[user_ratings.isna().any(axis=1)]

user_ratings.dropna(inplace=True)

print(user_ratings.isna().sum())

joke_id                 0
USF1000_rating        221
USF1001_rating         77
USF1002_rating          0
source                  0
datasource_joke_id      0
dtype: int64
joke_id               0
USF1000_rating        0
USF1001_rating        0
USF1002_rating        0
source                0
datasource_joke_id    0
dtype: int64


In [45]:
# Merge the joke text with the user ratings

def merge_joke_text(row):
    joke_id = row['datasource_joke_id']
    joke_source = row['source']

    if joke_source == 'reddit':
        joke_text = reddit_joke_data[reddit_joke_data['jokeId'] == joke_id]['jokeText'].values
        if joke_text:
            row['jokeText'] = joke_text[0]
        else:
            print ("Joke not found for reddit with id: " + joke_id)
        
    elif joke_source == 'jester':
        joke_text = jester_joke_data[jester_joke_data['jokeId'] == int(joke_id)]['jokeText'].values
        if joke_text:
            row['jokeText'] = joke_text[0]
        else:
            print ("Joke not found for jester with id->" + joke_id)
        
    elif joke_source == 'stupidstuff':
        joke_id = int(joke_id)
        joke_text = stupidstuff_jokes[stupidstuff_jokes['jokeId'] == joke_id]['jokeText'].values
        
        if joke_text:
            row['jokeText'] = joke_text[0]
        else:
            print ("Joke not found for stupidstuff with id: " + joke_id)

    return row


temp_df = user_ratings.apply(merge_joke_text, axis=1)

In [46]:
one_hot = pd.get_dummies(temp_df['source'])

temp_df = temp_df.join(one_hot)

joke_info_dataset = temp_df.drop('source',axis = 1)

In [47]:
users_info = pd.read_csv('datasets/users.csv')

users_info.head()

,user_id,name,age,gender,ethnicity,country,location,avg_user_rating,city,buddy
0,USF1000,Tanya,24,Female,South East Asian,India,San Francisco,5.968586,Delhi,USF1001
1,USF1001,rishab,26,Male,South East Asian,India,San Francisco,6.727749,Mumbai,USF1000
2,USF1002,Parisa,26,Female,Middle Eastern,Iran,San Francisco,0.806283,Ahvaz,USF1001


#### Working on Step 2 now

In [48]:
df = temp_df

df

,joke_id,USF1000_rating,USF1001_rating,USF1002_rating,source,datasource_joke_id,jokeText,jester,reddit,stupidstuff
0,RE5tz52q,1.0,8.0,0.0,reddit,5tz52q,I hate how you cant even say black paint anymo...,0,1,0
1,RE5tz4dd,1.0,8.0,0.0,reddit,5tz4dd,What's the difference between a Jew in Nazi Ge...,0,1,0
2,RE5tz319,0.0,9.0,8.0,reddit,5tz319,I recently went to America....\n...and being t...,0,1,0
3,RE5tz2wj,3.0,9.0,0.0,reddit,5tz2wj,"Brian raises his hand and says, “He’s in Heave...",0,1,0
4,RE5tz1pc,5.0,3.0,8.0,reddit,5tz1pc,You hear about the University book store worke...,0,1,0
...,...,...,...,...,...,...,...,...,...,...
136,SS2771,7.0,10.0,0.0,stupidstuff,2771,Q: How many Field Service Engineers does it ta...,0,0,1
137,SS1365,8.0,10.0,0.0,stupidstuff,1365,Yo mamma so dumb she sold her car for gasoline...,0,0,1
138,SS570,8.0,3.0,0.0,stupidstuff,570,"A woman gets home, schreeches her car into the...",0,0,1
139,SS431,7.0,7.0,0.0,stupidstuff,431,The psychology instructor had just finished a ...,0,0,1


In [49]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

tokenizer = AutoTokenizer.from_pretrained("unitary/toxic-bert")

model = AutoModelForSequenceClassification.from_pretrained("unitary/toxic-bert")
pipeline =  TextClassificationPipeline(model=model, tokenizer=tokenizer)

In [50]:
# and fill it with the predicted probability for each row
def assign_labels(model, df, label_names):
    for index, row in df.iterrows():
        text = row['jokeText']
        inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
        outputs = model(**inputs)
        logits = outputs.logits
        softmax = torch.softmax(logits, dim=1)
        predicted_probabilities = softmax.detach().numpy()[0]
        for i, label in enumerate(label_names):
            df.at[index, label] = predicted_probabilities[i]
    return df

In [51]:
for model_path in ['unitary/toxic-bert', 'michellejieli/emotion_text_classifier']:
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    label_names = list(model.config.id2label.values())
    df = assign_labels(model, df, label_names)
    

In [52]:
# MilaNLProc/bert-base-uncased-ear-misogyny
for model_path in ['MilaNLProc/bert-base-uncased-ear-misogyny', 'NLP-LTU/distilbert-sexism-detector']:
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    label_names = list(model.config.id2label.values())
    print(label_names)
    for index, row in df.iterrows():
        text = row['jokeText']
        inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
        outputs = model(**inputs)
        logits = outputs.logits
        softmax = torch.softmax(logits, dim=1)
        predicted_probabilities = softmax.detach().numpy()[0]
        for i, label in enumerate(label_names):
            if label in ['misogynist', 'sexist']:
                df.at[index, label] = predicted_probabilities[i]

['non-misogynist', 'misogynist']
['not sexist', 'sexist']


-------------------------------------------------------------------------------

This is where we calculate the metrics for every demographic

-------------------------------------------------------------------------------

In [53]:
users = pandas.read_csv('datasets/users.csv')

users_encoded = users.copy()
users_encoded = users_encoded.drop(columns=['name', 'user_id'], axis=1)
users_encoded['age'] = users['age'].astype(str)
users_encoded = pandas.get_dummies(users_encoded)

users_demographics = df.copy()
for index, row in df.iterrows():
    for col in users_encoded.columns:
        sum = 0
        # number_of_users is the number of rows in users_encoded
        number_of_users = len(users_encoded.index)
        for user_id in range(number_of_users):
            weight = users_encoded.at[user_id, col]
            rating = df.at[index, f'USF100{user_id}_rating']
            sum += weight * rating
        users_demographics.at[index, col] = sum / (number_of_users * 10)
users_demographics

,joke_id,USF1000_rating,USF1001_rating,USF1002_rating,source,datasource_joke_id,jokeText,jester,reddit,stupidstuff,...,ethnicity_Middle Eastern,ethnicity_South East Asian,country_India,country_Iran,location_San Francisco,city_Ahvaz,city_Delhi,city_Mumbai,buddy_USF1000,buddy_USF1001
0,RE5tz52q,1.0,8.0,0.0,reddit,5tz52q,I hate how you cant even say black paint anymo...,0,1,0,...,0.000000,0.300000,0.300000,0.000000,0.300000,0.000000,0.033333,0.266667,0.266667,0.033333
1,RE5tz4dd,1.0,8.0,0.0,reddit,5tz4dd,What's the difference between a Jew in Nazi Ge...,0,1,0,...,0.000000,0.300000,0.300000,0.000000,0.300000,0.000000,0.033333,0.266667,0.266667,0.033333
2,RE5tz319,0.0,9.0,8.0,reddit,5tz319,I recently went to America....\n...and being t...,0,1,0,...,0.266667,0.300000,0.300000,0.266667,0.566667,0.266667,0.000000,0.300000,0.300000,0.266667
3,RE5tz2wj,3.0,9.0,0.0,reddit,5tz2wj,"Brian raises his hand and says, “He’s in Heave...",0,1,0,...,0.000000,0.400000,0.400000,0.000000,0.400000,0.000000,0.100000,0.300000,0.300000,0.100000
4,RE5tz1pc,5.0,3.0,8.0,reddit,5tz1pc,You hear about the University book store worke...,0,1,0,...,0.266667,0.266667,0.266667,0.266667,0.533333,0.266667,0.166667,0.100000,0.100000,0.433333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,SS2771,7.0,10.0,0.0,stupidstuff,2771,Q: How many Field Service Engineers does it ta...,0,0,1,...,0.000000,0.566667,0.566667,0.000000,0.566667,0.000000,0.233333,0.333333,0.333333,0.233333
137,SS1365,8.0,10.0,0.0,stupidstuff,1365,Yo mamma so dumb she sold her car for gasoline...,0,0,1,...,0.000000,0.600000,0.600000,0.000000,0.600000,0.000000,0.266667,0.333333,0.333333,0.266667
138,SS570,8.0,3.0,0.0,stupidstuff,570,"A woman gets home, schreeches her car into the...",0,0,1,...,0.000000,0.366667,0.366667,0.000000,0.366667,0.000000,0.266667,0.100000,0.100000,0.266667
139,SS431,7.0,7.0,0.0,stupidstuff,431,The psychology instructor had just finished a ...,0,0,1,...,0.000000,0.466667,0.466667,0.000000,0.466667,0.000000,0.233333,0.233333,0.233333,0.233333


#### Step 3

In [54]:
df = pandas.read_json("datasets/joke_info.json")
USF_users = pandas.read_csv('datasets/users.csv')

avg_user_rating = []
for index, row in USF_users.iterrows():
    user_id = row['user_id']
    avg_user_rating.append(df[f'{user_id}_rating'].mean())

USF_users['avg_user_rating'] = avg_user_rating


In [55]:
USF1002_ratings = pandas.read_csv('datasets/ratings_USF1002.csv')
USF1002_ratings = USF1002_ratings[USF1002_ratings['joke_id'].str.startswith('JE')]

USF1000_ratings = pandas.read_csv('datasets/ratings_USF1000.csv')
USF1000_ratings = USF1000_ratings[USF1000_ratings['joke_id'].str.startswith('JE')]

USF1001_ratings = pandas.read_csv('datasets/ratings_USF1001.csv')
USF1001_ratings = USF1001_ratings[USF1001_ratings['joke_id'].str.startswith('JE')]

In [56]:
pred = pandas.DataFrame(columns=['joke_id', 'USF1002_rating', 'USF1000_rating', 'USF1001_rating'])
pred['joke_id'] = USF1002_ratings['joke_id']
pred['USF1002_rating'] = USF1002_ratings['rating']
pred['USF1000_rating'] = USF1000_ratings['rating']
pred['USF1001_rating'] = USF1001_ratings['rating']

In [57]:
extracted = pred.copy()
extracted = extracted.dropna()
extracted

,joke_id,USF1002_rating,USF1000_rating,USF1001_rating
39,JE40,1,1.0,9.0
40,JE41,0,1.0,5.0
41,JE42,0,0.0,10.0
42,JE43,0,3.0,0.0
43,JE44,0,5.0,10.0
44,JE45,0,4.0,10.0
45,JE46,0,0.0,10.0
46,JE47,0,2.0,10.0
47,JE48,3,3.0,10.0
48,JE49,0,0.0,8.0


In [58]:
# make a new  dataframe df2 that recalculates the ratings of all jokes by calculating the difference between the average rating of the user and the rating of the joke
# so for a user with id x in USF_users, the value of `x_rating` should be the difference between the average rating of the user from the USF_users dataframe and the rating of the joke from the df dataframe
# if a user has not rated any joke, the value of `x_rating` should be 0

df2 = df.copy()
for index, row in USF_users.iterrows():
    user_id = row['user_id']
    df2[f'{user_id}_rating'] =  row['avg_user_rating'] - df2[f'{user_id}_rating']

df2

,joke_id,USF1000_rating,USF1001_rating,USF1002_rating,datasource_joke_id,jokeText,jester,reddit,stupidstuff
0,RE5tz52q,5.10303,-1.206061,1.09697,5tz52q,I hate how you cant even say black paint anymo...,0,1,0
1,RE5tz4dd,5.10303,-1.206061,1.09697,5tz4dd,What's the difference between a Jew in Nazi Ge...,0,1,0
2,RE5tz319,6.10303,-2.206061,-6.90303,5tz319,I recently went to America....\n...and being t...,0,1,0
3,RE5tz2wj,3.10303,-2.206061,1.09697,5tz2wj,"Brian raises his hand and says, “He’s in Heave...",0,1,0
4,RE5tz1pc,1.10303,3.793939,-6.90303,5tz1pc,You hear about the University book store worke...,0,1,0
...,...,...,...,...,...,...,...,...,...
160,SS2771,-0.89697,-3.206061,1.09697,2771,Q: How many Field Service Engineers does it ta...,0,0,1
161,SS1365,-1.89697,-3.206061,1.09697,1365,Yo mamma so dumb she sold her car for gasoline...,0,0,1
162,SS570,-1.89697,3.793939,1.09697,570,"A woman gets home, schreeches her car into the...",0,0,1
163,SS431,-0.89697,-0.206061,1.09697,431,The psychology instructor had just finished a ...,0,0,1


In [59]:
similarity_matrix = pandas.DataFrame(index=USF_users['user_id'], columns=USF_users['user_id'])

for index, row in USF_users.iterrows():
    user_id = row['user_id']
    for index2, row2 in USF_users.iterrows():
        user_id2 = row2['user_id']
        similarity_matrix[user_id][user_id2] = (df2[f'{user_id}_rating'] * df2[f'{user_id2}_rating']).sum() / (numpy.sqrt((df2[f'{user_id}_rating']**2).sum()) * numpy.sqrt((df2[f'{user_id2}_rating']**2).sum()))

similarity_matrix

user_id,USF1000,USF1001,USF1002
user_id,,,
USF1000,1.0,0.257828,-0.109062
USF1001,0.257828,1.0,0.05376
USF1002,-0.109062,0.05376,1.0


### Calculate buddy here

In [60]:
USF_users['buddy'] = ''
for index, row in USF_users.iterrows():
    user_id = row['user_id']
    max_similarity = 0
    buddy = ''
    for index2, row2 in USF_users.iterrows():
        user_id2 = row2['user_id']
        if user_id2 != user_id and similarity_matrix[user_id][user_id2] > max_similarity:
            max_similarity = similarity_matrix[user_id][user_id2]
            buddy = user_id2
    USF_users.loc[index, 'buddy'] = buddy

USF_users

,user_id,name,age,gender,ethnicity,country,location,avg_user_rating,city,buddy
0,USF1000,Tanya,24,Female,South East Asian,India,San Francisco,6.103030,Delhi,USF1001
1,USF1001,rishab,26,Male,South East Asian,India,San Francisco,6.793939,Mumbai,USF1000
2,USF1002,Parisa,26,Female,Middle Eastern,Iran,San Francisco,1.096970,Ahvaz,USF1001


Step 4 from this point onwards

In [61]:
df = users_demographics

In [62]:
## This is our target user. Currently we're targeting jokes for Tanya -> USF1000

curr_user_traits = {
    "age": 24,
    "gender": "Female",
    "ethnicity": "South East Asian",
    "country": "India",
    "location" : "San Francisco"
}



# These traits tell us which columns to use from the user_demographics dataframe

col_names_to_use = [ str(key) + '_' + str(value) for key, value in curr_user_traits.items()]

In [63]:
# for col in col_names_to_use:
#     result_df = df[df[col] > 0.5]

In [80]:
columns_to_keep = col_names_to_use

columns_to_keep.extend([
    # 'USF1000_rating', 
    # 'USF1001_rating', 
    # 'USF1002_rating',
    # "jester",
    # "reddit",
    # "stupidstuff",
    # 'joke_id',
    # 'jokeText', 
    'jester', 
    'reddit', 
    'stupidstuff',
    'toxic', 
    'severe_toxic', 
    'obscene', 
    'threat', 
    'insult', 
    'identity_hate',
    'anger', 
    'disgust', 
    'fear', 
    'joy', 
    'neutral', 
    'sadness', 
    'surprise',
    'misogynist', 
    'sexist', 
])

# label_df = temp_df['USF1002_rating'].apply(lambda x: 1 if x > 0.5 else 0)

# result_df = result_df[result_df['USF1002_rating']]

# label_df = result_df['USF1002_rating']

# result_df = result_df[columns_to_keep]

## Since the target user is USF1000 now

label_df_demographics = df['USF1000_rating']

features_df_demographics = df[columns_to_keep]

In [81]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features_df_demographics, label_df_demographics , test_size=0.2, random_state=42)

In [82]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import GridSearchCV


# params = {'max_depth': [3, 4, 5], # Go smaller
#           'criterion': ['entropy', 'gini'],
#           'n_estimators': [115, 130, 150], # Go smaller, 3/5/10,x,
#           'max_features': [None]
#         }

# params = {'max_depth': [None, 5, 10, 20, 30], 
#           'criterion': ['absolute_error', 'squared_error'],
#           'n_estimators': [115, 130, 150, 200], 
#           'max_features': [None, 'log2']
#         }

params = {'max_depth': [5, 10, 9],
          'criterion': ['squared_error'],
          'n_estimators': [50, 150, 200]
        }


folds = 5
rf = RandomForestRegressor()

clf = GridSearchCV(rf, params, cv=folds, n_jobs=-1)

clf.fit(X_train, y_train)
print(clf.best_params_)

rf_demographics = RandomForestRegressor(**clf.best_params_, random_state=42, n_jobs=-1, oob_score=True)

rf_demographics.fit(X_train, y_train)
y_pred = rf_demographics.predict(X_test)

accuracy = rf_demographics.score(X_test, y_test)
print("Accuracy->", accuracy)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

-------------------------------------------------------------------------------
Step 5
-------------------------------------------------------------------------------
-------------------------------------------------------------------------------

In [85]:
jokes_df = users_demographics.copy()

In [88]:
## get a dict of target user traits

target_user = "USF1000"

target_user_traits = users[users['user_id'] == target_user].to_dict('records')[0]

## Get info of taget user's buddy

target_user_buddy_traits = users[users['user_id'] == target_user_traits['buddy']].to_dict('records')[0]

In [89]:
# Get columns to keep for our model 

relevant_columns = set()

traits_to_ignore = ['user_id', 'buddy', 'name', 'avg_user_rating', 'city']

for key, value in target_user_traits.items():
    if key not in traits_to_ignore:
        relevant_columns.add(str(key) + "_" + str(value))


for key, value in target_user_buddy_traits.items():
    if key not in traits_to_ignore:
        relevant_columns.add(str(key) + "_" + str(value))

relevant_columns = list(relevant_columns)

In [90]:
similarity_threshold = 0.30

for col in relevant_columns:
    jokes_df = jokes_df[jokes_df[col] > similarity_threshold]

In [91]:
original_relevant_columns = [
    target_user_traits['user_id'] + '_rating',
    # target_user_buddy_traits['user_id'] + '_rating',
    # 'joke_id',
    # 'datasource_joke_id',
    # 'jokeText',
    'jester', 
    'reddit', 
    'stupidstuff',
    'toxic', 
    'severe_toxic', 
    'obscene', 
    'threat', 
    'insult', 
    'identity_hate',
    'anger', 
    'disgust', 
    'fear', 
    'joy', 
    'neutral', 
    'sadness', 
    'surprise',
    'misogynist', 
    'sexist'
]

original_relevant_columns.extend(relevant_columns)

jokes_df = jokes_df[original_relevant_columns]

jokes_df.head()

,USF1000_rating,jester,reddit,stupidstuff,toxic,severe_toxic,obscene,threat,insult,identity_hate,...,surprise,misogynist,sexist,ethnicity_South East Asian,location_San Francisco,age_24,gender_Male,country_India,gender_Female,age_26
124,10.0,0,0,1,0.631412,0.005822,0.265192,0.005896,0.076584,0.015095,...,0.059827,0.003128,0.078934,0.666667,0.666667,0.333333,0.333333,0.666667,0.333333,0.333333
127,10.0,0,0,1,0.636481,0.047341,0.107221,0.051621,0.089494,0.067843,...,0.037257,0.000158,0.010693,0.666667,0.666667,0.333333,0.333333,0.666667,0.333333,0.333333
129,10.0,0,0,1,0.771947,0.001812,0.067323,0.002093,0.147464,0.009362,...,0.015873,0.000238,0.481255,0.666667,0.666667,0.333333,0.333333,0.666667,0.333333,0.333333
130,10.0,0,0,1,0.486036,0.007341,0.343611,0.003386,0.141612,0.018014,...,0.012207,0.000258,0.104133,0.666667,0.666667,0.333333,0.333333,0.666667,0.333333,0.333333
131,10.0,0,0,1,0.818475,0.000337,0.008088,0.000244,0.170631,0.002225,...,0.003500,0.999519,0.349303,0.666667,0.666667,0.333333,0.333333,0.666667,0.333333,0.333333


In [98]:
from sklearn.model_selection import train_test_split

label_df_buddy = jokes_df[target_user_traits['user_id'] + '_rating']
jokes_df = jokes_df.drop([target_user_traits['user_id'] + '_rating', target_user_traits['user_id'] + '_rating'], axis=1)

X_train_buddy, X_test_buddy, y_train_buddy, y_test_buddy = train_test_split(jokes_df, label_df_buddy , test_size=0.1, random_state=42)

KeyError: 'USF1000_rating'

In [99]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier


# params = {'max_depth': [3, 4, 5], # Go smaller
#           'criterion': ['entropy', 'gini'],
#           'n_estimators': [115, 130, 150], # Go smaller, 3/5/10,x,
#           'max_features': [None]
#         }

# params = {'max_depth': [None, 5, 10, 20, 30], 
#           'criterion': ['absolute_error', 'squared_error'],
#           'n_estimators': [115, 130, 150, 200], 
#           'max_features': [None, 'log2']
#         }

params = {'max_depth': [5, 10, 9],
          'criterion': ['squared_error'],
          'n_estimators': [50, 150, 200]
        }


folds = 5
rf_buddy = RandomForestRegressor()

clf = GridSearchCV(rf_buddy, params, cv=folds, n_jobs=-1)

clf.fit(X_train_buddy, y_train_buddy, **{'n_splits': 4})
print(clf.best_params_)

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

# Train the model with the best parameters obtained

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

rf = RandomForestRegressor(**clf.best_params_, random_state=42, n_jobs=-1)

rf.fit(X_train_buddy, y_train_buddy)
y_pred = rf.predict(X_test_buddy)

accuracy = rf.score(X_test_buddy, y_test_buddy)
print("Accuracy->", accuracy)

rmse = mean_squared_error(y_test_buddy, y_pred, squared=False)
print("RMSE->", rmse)

ValueError: Cannot have number of splits n_splits=5 greater than the number of samples: n_samples=4.

In [95]:
X_train_buddy

,jester,reddit,stupidstuff,toxic,severe_toxic,obscene,threat,insult,identity_hate,anger,...,surprise,misogynist,sexist,ethnicity_South East Asian,location_San Francisco,age_24,gender_Male,country_India,gender_Female,age_26
131,0,0,1,0.818475,0.000337,0.008088,0.000244,0.170631,0.002225,0.020293,...,0.003500,0.999519,0.349303,0.666667,0.666667,0.333333,0.333333,0.666667,0.333333,0.333333
129,0,0,1,0.771947,0.001812,0.067323,0.002093,0.147464,0.009362,0.937975,...,0.015873,0.000238,0.481255,0.666667,0.666667,0.333333,0.333333,0.666667,0.333333,0.333333
124,0,0,1,0.631412,0.005822,0.265192,0.005896,0.076584,0.015095,0.543601,...,0.059827,0.003128,0.078934,0.666667,0.666667,0.333333,0.333333,0.666667,0.333333,0.333333
130,0,0,1,0.486036,0.007341,0.343611,0.003386,0.141612,0.018014,0.026760,...,0.012207,0.000258,0.104133,0.666667,0.666667,0.333333,0.333333,0.666667,0.333333,0.333333
